In [ ]:
import sys

sys.path.append("../")
# import time
# time.sleep(7200)

In [ ]:
config = {
    "pretrained_model": "bert-base-uncased",
    # emilyalsentzer/Bio_ClinicalBERT ../resources/bert/biobert_v1.1_pubmed/
    "tokenizer": "bert-base-uncased",
    "max_seq_length": 256,
    "batch_size": 6,
    "lr": 2e-5,
    "epochs": 10,
    "device": "cuda",
    "gpu_ids": "2",
    "seed": 2020,
    "fp16": False,
    "loss_scale": 0,
    "gradient_accumulation_steps": 1,
    "warmup_proportion": 0.1,
    "num_labels": 8,
    "is_multilabel": True,
    "valid_metric": "micro_f1",
    "model_save_dir": "../checkpoints/bert_cased_256_bio_26k_20200908/",
    "patience": 4,
}

In [3]:
from beta_nlp.models.bert_cls import BertModel

cls = BertModel(config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [4]:
import numpy as np
import torch


def int2label(label, max_len=8):
    labels = []
    s = str(label)
    l_n = max_len - len(s)
    one_hot = np.array([0] * max_len)
    for i, c in enumerate(s):
        if c == "1":
            labels.append(l_n + i)
    one_hot[labels] = 1
    return list(one_hot)

In [5]:
import pandas as pd

train_set = pd.read_csv(
    "../datasets/litcovid/covid-classification/data/FullLitCovid/train.tsv",
    sep="\t",
    header=None,
)
train_set.rename(columns={1: "docs"}, inplace=True)
train_set.loc[:, "labels"] = train_set[0].apply(int2label)
train_set.head()

,0,docs,labels
0,1000,The Battle Against COVID-19 in Jordan: An Earl...,"[0, 0, 0, 0, 1, 0, 0, 0]"
1,1000,Restablecimiento de la actividad en los servic...,"[0, 0, 0, 0, 1, 0, 0, 0]"
2,1000,Impact of Severe Acute Respiratory Syndrome (S...,"[0, 0, 0, 0, 1, 0, 0, 0]"
3,1100000,Participation of the IL-10RB Related Cytokines...,"[0, 1, 1, 0, 0, 0, 0, 0]"
4,1100000,[Inhibitors of RAS Might Be a Good Choice for ...,"[0, 1, 1, 0, 0, 0, 0, 0]"


In [6]:
dev_set = pd.read_csv(
    "../datasets/litcovid/covid-classification/data/FullLitCovid/dev.tsv",
    sep="\t",
    header=None,
)
dev_set.rename(columns={1: "docs"}, inplace=True)
dev_set.loc[:, "labels"] = dev_set[0].apply(int2label)
dev_set.head()

,0,docs,labels
0,1000,COVID-19 Pandemic and Its Effects on Cancer Ca...,"[0, 0, 0, 0, 1, 0, 0, 0]"
1,11000000,Association of Padua prediction score with in-...,"[1, 1, 0, 0, 0, 0, 0, 0]"
2,10000000,Decreased B cells on admission was associated ...,"[1, 0, 0, 0, 0, 0, 0, 0]"
3,1000,Decline in Child Vaccination Coverage During t...,"[0, 0, 0, 0, 1, 0, 0, 0]"
4,1000,Inequity and the disproportionate impact of CO...,"[0, 0, 0, 0, 1, 0, 0, 0]"


In [7]:
test_set = pd.read_csv(
    "../datasets/litcovid/covid-classification/data/FullLitCovid/test.tsv",
    sep="\t",
    header=None,
)
test_set.rename(columns={1: "docs"}, inplace=True)
test_set.loc[:, "labels"] = test_set[0].apply(int2label)
test_set.head()

,0,docs,labels
0,11000000,SARS-CoV-2 infection in cancer patients underg...,"[1, 1, 0, 0, 0, 0, 0, 0]"
1,10000000,Diagnosis of COVID-19 infection in children: l...,"[1, 0, 0, 0, 0, 0, 0, 0]"
2,1000,Lessons Learned from COVID-19 Pandemic in Ital...,"[0, 0, 0, 0, 1, 0, 0, 0]"
3,10001000,Universal screening for SARS-CoV-2 in pregnant...,"[1, 0, 0, 0, 1, 0, 0, 0]"
4,1000000,Passive Immunity for Coronavirus Disease 2019:...,"[0, 1, 0, 0, 0, 0, 0, 0]"


In [8]:
# train_set = data_df[data_df["flag"]=="train"]
# dev_set = data_df[data_df["flag"]=="valid"]
# test_set = data_df[data_df["flag"]=="test"]

In [9]:
len(train_set.index), len(dev_set.index), len(test_set.index)

(16126, 2305, 4607)

In [ ]:
# train_set = train_set.iloc[:100]
# dev_set = dev_set.iloc[:100]

cls.train(train_set, dev_set)


Execute [train_an_epoch] method costing 135732.78 ms
[Epoch 0] loss: 1861.9187163710594


/home/zm324/anaconda3/envs/doc_cls/lib/python3.8/site-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")



Execute [eval] method costing 19338.63 ms
--------------------------------------------------------------------------------
[Epoch 0]performance on validation set
+----+-----------------+-----------+
|    | metrics         |    values |
|----+-----------------+-----------|
|  0 | accuracy        | 0         |
|  1 | macro_precision | 0.122758  |
|  2 | macro_recall    | 0.396816  |
|  3 | macro_f1        | 0.0930695 |
|  4 | micro_precision | 0.112773  |
|  5 | micro_recall    | 0.276117  |
|  6 | micro_f1        | 0.16014   |
+----+-----------------+-----------+
--------------------------------------------------------------------------------


In [ ]:
result = cls.test(dev_set)
result

In [ ]:
import csv

with open("../results/bert_cased_256_bio_26k_20200908_test.csv", "w") as f:
    for key in result.keys():
        f.write("%s,%s\n" % (key, result[key]))

In [ ]:
result

In [ ]:
data_url = "/home/zm324/workspace/data/med_cls/promed_26k/"
df = pd.read_csv(data_dir + "train.tsv")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/zm324/anaconda3/envs/doc_cls/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-916d1ee3ada8>", line 2, in <module>
    df = pd.read_csv(os.path.join(data_dir + "train.tsv"))
NameError: name 'pd' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/zm324/anaconda3/envs/doc_cls/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/zm324/anaconda3/envs/doc_cls/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1170, in get_records
    return _fixed_getinnerframes(etb, number_of_l